<a href="https://colab.research.google.com/github/fstrike7/migracion-tecnologica-DW/blob/main/TPF_Visualizacion_Grupo7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabajo Práctico Final - Visualización de Información
## Visualizando datos en un notebook.
**Grupo 7.**  
**Docente:** Diego Ariel Aizemberg  
**Integrantes:** Faustino Beatrice, Daniel Simosa y Diego Romero



## Índice
1. [Consigna 1 — Exploración de Datos (EDA)](#consigna1)
2. [Consigna 2 — Planteamiento de preguntas de investigación](#consigna2)
3. [Consigna 3 — Visualizaciones e interpretaciones](#consigna3)
4. [Conclusiones y próximos pasos](#conclusiones)


> **Contexto del dataset (resumen):**  
El trabajo se basa en el caso de **optimización del uso de red móvil y migración tecnológica en CABA**, con foco en la transición desde 2G hacia 4G/5G. Se proponen análisis por comuna/barrio, tecnología, dispositivo y tiempo, de modo de caracterizar zonas críticas, usuarios no migrados e impacto de la migración.
Se puede visitar el proyecto para la materia de DW & OLAP en este link: https://github.com/fstrike7/migracion-tecnologica-DW

> **campos esperados en el dataset:**
- `fecha` (YYYY-MM-DD), `anio`, `mes`, `nombre_mes`, `hora`
- `partido_comuna`, `barrio`, `id_celda`
- `tecnologia` (e.g., 2G/3G/4G/5G)
- `id_usuario`, `id_device`, `modelo`, `marca`, `gama`, `compatible_lte`, `compatible_5g`
- Métricas de estado: `service_status`, `release_cause1`, `release_cause2`


## Consigna 1 — Exploración de Datos (EDA) <a id="consigna1"></a>

En esta sección se carga el dataset, se realiza limpieza/preparación y un análisis exploratorio inicial:
- Descripción de distribuciones.
- Detección de valores atípicos.
- Análisis de correlaciones.
- Verificación de valores faltantes.
- Documentación de hallazgos relevantes.


In [22]:
# === Configuración e imports ===
import pandas as pd
import numpy as np
import altair as alt
alt.data_transformers.disable_max_rows()


import pandas as pd

BASE = "https://raw.githubusercontent.com/fstrike7/migracion-tecnologica-DW/main/data"

def read(name):
    url = f"{BASE}/{name}"
    return pd.read_csv(url, encoding="latin1")

# --- Cargar dimensiones ---
dim_celda          = read("dim_celda.csv")
dim_device         = read("dim_device.csv")
dim_fecha          = read("dim_fecha.csv")
dim_service_status = read("dim_service_status.csv")
dim_tiempo         = read("dim_tiempo.csv")
dim_usuario        = read("dim_usuario.csv")

# --- Cargar hechos ---
f_status           = read("f_status.csv")
f_registraciones   = read("f_registraciones.csv")

# vista de info basica
print("f_status.shape ->", f_status.shape)
display(f_status.head())
display(f_status.tail())


f_status.shape -> (200000, 8)


,id,id_fecha,id_tiempo,timestamp,access_type,id_service_status,id_usuario,id_celda
0,1,20250516,141,2025-05-16 00:01:41,3G,24,21788,WCN275X
1,2,20250516,142,2025-05-16 00:01:42,3G,17,79404,WBM080X
2,3,20250516,142,2025-05-16 00:01:42,3G,24,42913,WBO003Z
3,4,20250516,143,2025-05-16 00:01:43,2G,17,70339,CBC070O
4,5,20250516,148,2025-05-16 00:01:48,3G,24,100596,WBA018Z


,id,id_fecha,id_tiempo,timestamp,access_type,id_service_status,id_usuario,id_celda
199995,199996,20250516,2753,2025-05-16 00:27:53,2G,21,2884,VCN454Y
199996,199997,20250516,2754,2025-05-16 00:27:54,2G,21,2974,CUE049C
199997,199998,20250516,2755,2025-05-16 00:27:55,2G,21,2972,CFN003O
199998,199999,20250516,2755,2025-05-16 00:27:55,2G,21,2988,GES021C
199999,200000,20250516,2758,2025-05-16 00:27:58,2G,21,2986,CDE030B


In [23]:

# === Construcción de df_main a partir de hechos + dimensiones ===
import pandas as pd

def _has_cols(df, cols):
    return all(c in df.columns for c in cols)

df_main = f_status.copy()

# Join con dimensiones si existen las keys
if 'id_celda' in df_main.columns and 'id_celda' in dim_celda.columns:
    df_main = df_main.merge(dim_celda, on='id_celda', how='left')
if 'id_device' in df_main.columns and 'id_device' in dim_device.columns:
    df_main = df_main.merge(dim_device, on='id_device', how='left')
if 'id_service_status' in df_main.columns and 'id_service_status' in dim_service_status.columns:
    df_main = df_main.merge(dim_service_status, on='id_service_status', how='left')
if 'id_fecha' in df_main.columns and 'id_fecha' in dim_fecha.columns:
    df_main = df_main.merge(dim_fecha, on='id_fecha', how='left')
if 'id_tiempo' in df_main.columns and 'id_tiempo' in dim_tiempo.columns:
    df_main = df_main.merge(dim_tiempo, on='id_tiempo', how='left')

# Parse de fecha si existe
for col in ('fecha', 'Fecha', 'FECHA'):
    if col in df_main.columns:
        df_main['fecha'] = pd.to_datetime(df_main[col], errors='coerce')
        break

# Normalización de columnas "estándar" esperadas por el notebook
def alias_col(df, candidates, new_name):
    for c in candidates:
        if c in df.columns:
            if new_name not in df.columns:
                df[new_name] = df[c]
            return
# partido_comuna puede venir como 'partido_comuna' / 'comuna' / 'barrio' / 'localidad'
alias_col(df_main, ['partido_comuna','comuna','barrio','localidad'], 'partido_comuna')
# tecnologia podría existir en f_status o alguna dim; si existe, lo dejamos como está
# service_status puede ser 'service_status' o 'status' o 'estado'
alias_col(df_main, ['service_status','status','estado'], 'service_status')
# hora puede venir de dim_tiempo ('hora','hour','hh')
alias_col(df_main, ['hora','hour','hh'], 'hora')
# compat flags si existen
for src, dst in [(['compatible_lte','lte_compatible','is_lte'], 'compatible_lte'),
                 (['compatible_5g','is_5g','5g_compatible'], 'compatible_5g')]:
    alias_col(df_main, src, dst)

print("df_main.shape ->", df_main.shape)
display(df_main.head(5))


df_main.shape -> (200000, 40)


,id,id_fecha,id_tiempo,timestamp,access_type,id_service_status,id_usuario,id_celda,sitio,tecnologia,...,semana_nro,dia_semana_nro,dia_semana,dia_semana_corto,trimestre,año_trimestre,tiempo,hora,minuto,segundo
0,1,20250516,141,2025-05-16 00:01:41,3G,24,21788,WCN275X,CN275,3G,...,20,6,viernes,vier,Q2,2025/Q2,00:01:41,0,1,41
1,2,20250516,142,2025-05-16 00:01:42,3G,17,79404,WBM080X,BM080,3G,...,20,6,viernes,vier,Q2,2025/Q2,00:01:42,0,1,42
2,3,20250516,142,2025-05-16 00:01:42,3G,24,42913,WBO003Z,BO003,3G,...,20,6,viernes,vier,Q2,2025/Q2,00:01:42,0,1,42
3,4,20250516,143,2025-05-16 00:01:43,2G,17,70339,CBC070O,BC070,2G,...,20,6,viernes,vier,Q2,2025/Q2,00:01:43,0,1,43
4,5,20250516,148,2025-05-16 00:01:48,3G,24,100596,WBA018Z,BA018,3G,...,20,6,viernes,vier,Q2,2025/Q2,00:01:48,0,1,48


In [24]:
# === Valores faltantes ===
if not f_status.empty:
    missing = f_status.isna().sum().sort_values(ascending=False)
    missing = missing[missing > 0]
    display(missing)
else:
    print("El DataFrame está vacío (placeholder). Cargue el dataset en DATA_PATH.")

,0


In [25]:
# === Duplicados ===
if not df_main.empty:
    print("Registros duplicados:", df_main.duplicated().sum())
else:
    print("El DataFrame está vacío (placeholder).")

Registros duplicados: 0


In [26]:
# === Distribuciones (describe) ===
if not df_main.empty:
    display(df_main.describe(include='all'))
else:
    print("El DataFrame está vacío (placeholder).")

,id,id_fecha,id_tiempo,timestamp,access_type,id_service_status,id_usuario,id_celda,sitio,tecnologia,...,semana_nro,dia_semana_nro,dia_semana,dia_semana_corto,trimestre,año_trimestre,tiempo,hora,minuto,segundo
count,200000.000000,200000.0,200000.00000,200000,200000,200000.000000,200000.000000,200000,200000,200000,...,200000.0,200000.0,200000,200000,200000,200000,200000,200000.0,200000.000000,200000.000000
unique,NaN,NaN,NaN,1006,2,NaN,NaN,20323,5184,3,...,NaN,NaN,1,1,1,1,1006,NaN,NaN,NaN
top,NaN,NaN,NaN,2025-05-16 00:16:22,3G,NaN,NaN,WBH057Z,BH057,3G,...,NaN,NaN,viernes,vier,Q2,2025/Q2,00:16:22,NaN,NaN,NaN
freq,NaN,NaN,NaN,509,178839,NaN,NaN,353,1281,175282,...,NaN,NaN,200000,200000,200000,200000,509,NaN,NaN,NaN
mean,100000.500000,20250516.0,1374.33085,NaN,NaN,24.507130,64984.188490,NaN,NaN,NaN,...,20.0,6.0,NaN,NaN,NaN,NaN,NaN,0.0,13.445535,29.777350
min,1.000000,20250516.0,141.00000,NaN,NaN,1.000000,1.000000,NaN,NaN,NaN,...,20.0,6.0,NaN,NaN,NaN,NaN,NaN,0.0,1.000000,0.000000
25%,50000.750000,20250516.0,1011.00000,NaN,NaN,24.000000,36198.750000,NaN,NaN,NaN,...,20.0,6.0,NaN,NaN,NaN,NaN,NaN,0.0,10.000000,14.000000
50%,100000.500000,20250516.0,1308.00000,NaN,NaN,24.000000,68131.000000,NaN,NaN,NaN,...,20.0,6.0,NaN,NaN,NaN,NaN,NaN,0.0,13.000000,30.000000
75%,150000.250000,20250516.0,1650.00000,NaN,NaN,24.000000,97363.000000,NaN,NaN,NaN,...,20.0,6.0,NaN,NaN,NaN,NaN,NaN,0.0,16.000000,45.000000
max,200000.000000,20250516.0,2758.00000,NaN,NaN,37.000000,116765.000000,NaN,NaN,NaN,...,20.0,6.0,NaN,NaN,NaN,NaN,NaN,0.0,27.000000,59.000000


In [30]:
# === Correlaciones (variables numéricas) ===
if not df_main.empty:
    num_cols = df_main.select_dtypes(include=np.number).columns
    if len(num_cols) >= 2:
        corr = df_main[num_cols].corr(numeric_only=True)
        corr_reset = corr.reset_index().melt('index')
        corr_reset.columns = ['var_x', 'var_y', 'correlacion']
        chart = alt.Chart(corr_reset).mark_rect().encode(
            x=alt.X('var_x:O', sort=None, title='Variable X'),
            y=alt.Y('var_y:O', sort=None, title='Variable Y'),
            tooltip=['var_x', 'var_y', alt.Tooltip('correlacion:Q', format='.2f')]
        ).properties(title='Matriz de correlación (numéricas)')
        display(chart)
    else:
        print("No hay suficientes variables numéricas para calcular correlaciones.")
else:
    print("El DataFrame está vacío (placeholder).")

alt.Chart(...)

In [32]:
# === Detección de outliers (robusta, IQR) ===
import numpy as np, pandas as pd, re

if not df_main.empty:
    df_num = df_main.select_dtypes(include=np.number).copy()
    if df_num.shape[1] == 0:
        print("No hay columnas numéricas para evaluar outliers.")
    else:
        n = len(df_num)
        id_like = [c for c in df_num.columns if re.match(r'(^id_.*|.*_id$|^id$)', str(c), flags=re.IGNORECASE)]
        low_var, few_unique = [], []
        for c in df_num.columns:
            if c in id_like:
                continue
            std = float(df_num[c].std(skipna=True))
            if not np.isfinite(std) or std < 1e-12:
                low_var.append(c); continue
            if df_num[c].nunique(dropna=True) <= max(10, int(0.02*n)):
                few_unique.append(c)
        exclude = set(id_like) | set(low_var) | set(few_unique)
        cols = [c for c in df_num.columns if c not in exclude]
        print(f"Columnas analizadas ({len(cols)}):", cols[:30], "..." if len(cols) > 30 else "")
        if exclude:
            print("Columnas excluidas:", {"id_like": id_like, "low_var": low_var, "few_unique": few_unique})
        if len(cols) == 0:
            print("No quedan columnas numéricas adecuadas para evaluar outliers.")
        else:
            outlier_counts = {}
            for c in cols:
                Q1, Q3 = df_num[c].quantile(0.25), df_num[c].quantile(0.75)
                IQR = Q3 - Q1
                if IQR == 0 or not np.isfinite(IQR):
                    continue
                lower, upper = Q1 - 1.5*IQR, Q3 + 1.5*IQR
                mask = (df_num[c] < lower) | (df_num[c] > upper)
                outlier_counts[c] = int(mask.sum())
            if outlier_counts:
                display(pd.Series(outlier_counts).sort_values(ascending=False).pipe(lambda s: s[s>0]))
            else:
                print("No se detectaron outliers con IQR en las columnas analizadas.")
else:
    print("El DataFrame está vacío (placeholder).")


Columnas analizadas (0): [] 
Columnas excluidas: {'id_like': ['id', 'id_fecha', 'id_tiempo', 'id_service_status', 'id_usuario'], 'low_var': ['año', 'mes', 'dia', 'semana_nro', 'dia_semana_nro', 'hora', 'anio'], 'few_unique': ['comuna', 'minuto', 'segundo']}
No quedan columnas numéricas adecuadas para evaluar outliers.


**Hallazgos EDA (completar):**

Sobre outliers

- No se detectaron columnas numéricas adecuadas para el análisis de outliers, ya que la mayoría corresponden a identificadores, variables de calendario de baja varianza o categóricas codificadas numéricamente (ej. comuna, tech_order).

- Esto implica que no aparecen outliers numéricos relevantes en el dataset, lo cual es consistente dado que no hay variables continuas como métricas de tráfico, caudal o volumen de datos.”

Sobre la naturaleza del dataset

- Las variables numéricas presentes cumplen funciones de identificación (claves foráneas) o discretización temporal, más que métricas continuas. Por lo tanto, el análisis de outliers no aporta valor en este caso.


## Consigna 2 — Planteamiento de preguntas de investigación <a id="consigna2"></a>

A continuación, se formulan preguntas significativas a responder con el dataset seleccionado. Estas preguntas se **basan en el trabajo de DW & OLAP** del grupo (optimización del uso de red móvil y migración tecnológica en CABA).


**Preguntas (adaptadas del trabajo DW & OLAP):**
1. **Identificación de zonas críticas**  
   - ¿Qué porcentaje de conexiones por comuna/barrio se realiza aún en redes 2G?
   - ¿Cómo se distribuyen las antenas por tipo de red (2G, 4G, 5G) y proveedor en CABA?
2. **Caracterización de usuarios no migrados**  
   - ¿Qué porcentaje de dispositivos conectados a 2G no son compatibles con LTE (4G) o 5G por comuna?
3. **Impacto de la migración**  
   - De los clientes que pasan a 4G/5G, ¿cuántos no vuelven a conectarse a 2G en el período posterior?  
   - ¿Cómo evolucionó la tasa de fallas de conexión de las celdas antes y después de la migración?  
   - ¿En qué franjas horarias se registra mayor saturación de red en zonas con coexistencia 2G/4G?


## Consigna 3 — Visualizaciones e interpretaciones <a id="consigna3"></a>

Se construyen visualizaciones para responder a las preguntas. Cada visual se acompaña de una **breve interpretación**.


In [33]:
# === Normalización de columnas de tiempo para P1 ===
import pandas as pd

# 1) anio
if 'anio' not in df_main.columns:
    if 'año' in df_main.columns:
        df_main['anio'] = df_main['año']
    elif 'fecha' in df_main.columns:
        df_main['anio'] = pd.to_datetime(df_main['fecha'], errors='coerce').dt.year

# 2) nombre_mes
MESES_ES = {
    1: 'enero', 2: 'febrero', 3: 'marzo', 4: 'abril',
    5: 'mayo', 6: 'junio', 7: 'julio', 8: 'agosto',
    9: 'septiembre', 10: 'octubre', 11: 'noviembre', 12: 'diciembre'
}

if 'nombre_mes' not in df_main.columns:
    if 'mes' in df_main.columns:
        # si 'mes' es numérico (1-12)
        df_main['nombre_mes'] = pd.to_numeric(df_main['mes'], errors='coerce').map(MESES_ES)
    elif 'fecha' in df_main.columns:
        m = pd.to_datetime(df_main['fecha'], errors='coerce').dt.month
        df_main['nombre_mes'] = m.map(MESES_ES)

# 3) alias de comuna si hace falta
if 'partido_comuna' not in df_main.columns and 'comuna' in df_main.columns:
    df_main['partido_comuna'] = df_main['comuna']

# 4) ordenar meses correctamente (categoría ordenada)
if 'nombre_mes' in df_main.columns:
    orden_meses = list(MESES_ES.values())
    df_main['nombre_mes'] = (
        df_main['nombre_mes']
        .str.lower()
        .astype(pd.CategoricalDtype(categories=orden_meses, ordered=True))
    )



In [34]:
# === P1: Porcentaje de conexiones por tecnología y comuna/mes ===
if not df_main.empty:
    required_cols = {'anio', 'nombre_mes', 'partido_comuna', 'tecnologia'}
    if required_cols.issubset(df_main.columns):
        grouped = (
            df_main[df_main['tecnologia'].isin(['2G','3G','4G','5G'])]
            .groupby(['anio', 'nombre_mes', 'partido_comuna', 'tecnologia'], as_index=False)
            .size()
            .rename(columns={'size':'total'})
        )
        # % dentro de cada (anio, nombre_mes, partido_comuna)
        grouped['porcentaje'] = grouped.groupby(['anio','nombre_mes','partido_comuna'])['total']\
                                       .transform(lambda x: (x / x.sum()) * 100)

        chart = alt.Chart(grouped).mark_bar().encode(
            x=alt.X('tecnologia:N', title='Tecnología'),
            y=alt.Y('porcentaje:Q', title='% de conexiones'),
            color='tecnologia:N',
            column=alt.Column('partido_comuna:N', title='Comuna/Barrio'),
            tooltip=['anio','nombre_mes','partido_comuna','tecnologia',
                     alt.Tooltip('porcentaje:Q', format='.2f')]
        ).properties(title='% de conexiones por tecnología y comuna')
        display(chart)
    else:
        print("Faltan columnas para esta visualización:", required_cols - set(df_main.columns))
else:
    print("El DataFrame está vacío (placeholder).")


/tmp/ipython-input-772271826.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(['anio', 'nombre_mes', 'partido_comuna', 'tecnologia'], as_index=False)
/tmp/ipython-input-772271826.py:12: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped['porcentaje'] = grouped.groupby(['anio','nombre_mes','partido_comuna'])['total']\


alt.Chart(...)

**Interpretación (completar):**  
- [ ] Comunas con mayor dependencia de 2G: Atrueco, Ñorquinco por ejemplo.
- [ ] Comunas con mayor adopción 3G: Avellaneda, Vicente Lopez, Zarate.


In [37]:
# === Inferir flags de compatibilidad por device a partir de la tecnología usada ===
import pandas as pd

# 0) Alias por si falta partido_comuna
if 'partido_comuna' not in df_main.columns and 'comuna' in df_main.columns:
    df_main['partido_comuna'] = df_main['comuna']

# 1) Si las columnas ya existen, no hacemos nada
need_lte = 'compatible_lte' not in df_main.columns
need_5g  = 'compatible_5g'  not in df_main.columns

if (need_lte or need_5g):
    key = None
    # Tomamos la granularidad correcta para inferir: device si está, sino usuario
    if 'id_device' in df_main.columns:
        key = 'id_device'
    elif 'id_usuario' in df_main.columns:
        key = 'id_usuario'

    if key is not None and 'tecnologia' in df_main.columns:
        tech_by_key = (
            df_main[[key, 'tecnologia']]
            .dropna()
            .assign(t=lambda x: x['tecnologia'].astype(str).str.upper())
            .groupby(key)['t'].agg(lambda s: set(s))
            .reset_index()
        )
        if need_lte:
            tech_by_key['compatible_lte'] = tech_by_key['t'].apply(lambda ss: ('4G' in ss) or ('5G' in ss))
        if need_5g:
            tech_by_key['compatible_5g']  = tech_by_key['t'].apply(lambda ss: ('5G' in ss))

        # Merge de vuelta a df_main
        cols = [key] + [c for c in ['compatible_lte','compatible_5g'] if c in tech_by_key.columns]
        df_main = df_main.merge(tech_by_key[cols], on=key, how='left')

        # Si quedó NaN (no hay observaciones), tomar False
        if 'compatible_lte' in df_main.columns:
            df_main['compatible_lte'] = df_main['compatible_lte'].fillna(False)
        if 'compatible_5g' in df_main.columns:
            df_main['compatible_5g']  = df_main['compatible_5g'].fillna(False)
    else:
        # Si no podemos inferir (falta id_device/id_usuario o tecnologia), crear flags en False para que la celda P2 corra
        if need_lte:
            df_main['compatible_lte'] = False
        if need_5g:
            df_main['compatible_5g']  = False


In [38]:
# === P2: % de dispositivos no compatibles con LTE/5G por comuna ===
if not df_main.empty:
    required_cols = {'partido_comuna', 'compatible_lte', 'compatible_5g'}
    if required_cols.issubset(df_main.columns):
        # No compatible si (lte == False) y (5g == False)
        df_main['no_compatible_moderno'] = (~df_main['compatible_lte'].astype(bool)) & (~df_main['compatible_5g'].astype(bool))

        by_comuna = df_main.groupby('partido_comuna', as_index=False, observed=True).agg(
            total=('no_compatible_moderno', 'size'),
            no_compatibles=('no_compatible_moderno', 'sum')
        )
        by_comuna['porcentaje_no_compat'] = (by_comuna['no_compatibles'] / by_comuna['total']) * 100

        chart = alt.Chart(by_comuna).mark_bar().encode(
            x=alt.X('partido_comuna:N', sort='-y', title='Comuna/Barrio'),
            y=alt.Y('porcentaje_no_compat:Q', title='% no compatibles (LTE/5G)'),
            tooltip=['partido_comuna', alt.Tooltip('porcentaje_no_compat:Q', format='.2f')]
        ).properties(title='% de dispositivos no compatibles por comuna')
        display(chart)
    else:
        print("Faltan columnas para esta visualización:", required_cols - set(df_main.columns))
else:
    print("El DataFrame está vacío (placeholder).")


alt.Chart(...)

**Interpretación (completar):**  
- [ ] Comunas con mayor porcentaje de dispositivos no compatibles: Comuna 1, Adolfo Gonzalez Chaves.
- [ ] Posibles causas (gama, ingresos, antigüedad de dispositivos): Falta de inversión podría ser.
- [ ] Implicancias para políticas de migración (beneficios/planes): Planes de financiación que permitan cambiar de dispositivo.


In [45]:
# === P3a: Clientes que migran a 4G/5G y no regresan a 2G ===
if not df_main.empty:
    required_cols = {'id_usuario', 'fecha', 'tecnologia'}
    if required_cols.issubset(df_main.columns):
        # Normalizar fecha y tecnologia
        df_main['fecha'] = pd.to_datetime(df_main['fecha'], errors='coerce')
        df_main['tecnologia_norm'] = df_main['tecnologia'].astype(str).str.strip().str.upper()

        # Usuarios con al menos un uso de 4G/5G
        modern = df_main[df_main['tecnologia_norm'].isin(['4G','5G'])]
        first_modern = (modern.sort_values('fecha')
                        .groupby('id_usuario', as_index=False)
                        .first()[['id_usuario','fecha']])
        first_modern.columns = ['id_usuario','fecha_first_modern']

        merged = df_main.merge(first_modern, on='id_usuario', how='left')

        has_modern = merged[~merged['fecha_first_modern'].isna()]['id_usuario'].unique()

        # Revisar si después de esa fecha hubo conexiones 2G
        after = merged[merged['fecha'] > merged['fecha_first_modern']]
        returned_to_2g = after[after['tecnologia_norm'] == '2G']['id_usuario'].unique()

        total_migrated = len(has_modern)
        no_return = total_migrated - len(np.intersect1d(has_modern, returned_to_2g))

        print("Usuarios con uso 4G/5G (al menos una vez):", total_migrated)
        print("De ellos, NO regresaron a 2G posteriormente:", no_return)
        if total_migrated > 0:
            print("Porcentaje:", round((no_return/total_migrated)*100, 2), "%")
    else:
        print("Faltan columnas para este análisis:", required_cols - set(df_main.columns))
else:
    print("El DataFrame está vacío (placeholder).")

print("Como conclusión, la migración es un éxito ya que los usuarios en su totalidad no vuelven.")

Usuarios con uso 4G/5G (al menos una vez): 2075
De ellos, NO regresaron a 2G posteriormente: 2075
Porcentaje: 100.0 %
Como conclusión, la migración es un éxito ya que los usuarios en su totalidad no vuelven.


In [50]:
# === P3b: Tasa de fallas por hora en la fecha disponible ===
if not df_main.empty:
    required_cols = {'hora', 'service_status'}
    if required_cols.issubset(df_main.columns):
        s = df_main['service_status'].astype(str).str.upper()
        FAIL_TOKENS = {'FAIL','FAILED','ERROR','DROP','KO','FALLA','DOWN','TIMEOUT'}
        df_main['falla'] = s.isin(FAIL_TOKENS).astype(int)

        by_hour = df_main.groupby('hora', observed=True).agg(
            total=('falla','size'),
            fails=('falla','sum')
        ).reset_index()
        by_hour['tasa_falla'] = by_hour['fails'] / by_hour['total']

        chart = alt.Chart(by_hour).mark_line(point=True).encode(
            x=alt.X('hora:O', title='Hora del día'),
            y=alt.Y('tasa_falla:Q', title='Tasa de fallas'),
            tooltip=['hora', alt.Tooltip('tasa_falla:Q', format='.2f')]
        ).properties(title='Tasa de fallas por hora (16/05/2025)')
        display(chart)
    else:
        print("Faltan columnas: {'hora','service_status'}")

        # TODO: REVISAR POR QUE NO ESTA TRAYENDO BIEN LA HORA


alt.Chart(...)

In [51]:
# === P3c: Conexiones por hora en zonas 2G/4G — Global + Top comunas ===
import pandas as pd
import altair as alt

# Por si Altair se queja del tamaño
alt.data_transformers.disable_max_rows()

if not df_main.empty:
    required = {'hora', 'partido_comuna', 'tecnologia'}
    if required.issubset(df_main.columns):
        # Filtramos solo 2G/4G y armamos agregaciones por hora/comuna
        df_tmp = df_main.copy()
        df_tmp['tec_norm'] = df_tmp['tecnologia'].astype(str).str.strip().str.upper()
        subset = df_tmp[df_tmp['tec_norm'].isin(['2G','4G'])].copy()

        by_hour = (
            subset.groupby(['hora','partido_comuna'], as_index=False, observed=True)
                  .size().rename(columns={'size':'conexiones'})
        )

        # --- (1) Global: total de conexiones por hora (todas las comunas) ---
        by_hour_total = by_hour.groupby('hora', as_index=False, observed=True)['conexiones'].sum()

        chart_global = alt.Chart(by_hour_total).mark_line(point=True).encode(
            x=alt.X('hora:O', title='Hora del día'),
            y=alt.Y('conexiones:Q', title='Total de conexiones'),
            tooltip=[alt.Tooltip('hora:O'), alt.Tooltip('conexiones:Q')]
        ).properties(title='Conexiones totales por hora en zonas con 2G/4G')

        display(chart_global)

        # --- (2) Detalle: Top N comunas por conexiones totales ---
        N_COMUNAS = 10  # cambiá este valor si querés más/menos comunas
        top_comunas = (
            by_hour.groupby('partido_comuna', observed=True)['conexiones']
                   .sum().sort_values(ascending=False).head(N_COMUNAS).index
        )
        by_hour_top = by_hour[by_hour['partido_comuna'].isin(top_comunas)].copy()

        chart_top = alt.Chart(by_hour_top).mark_line(point=True).encode(
            x=alt.X('hora:O', title='Hora del día'),
            y=alt.Y('conexiones:Q', title='Conexiones'),
            color=alt.Color('partido_comuna:N', title='Comuna/Barrio'),
            tooltip=['partido_comuna', alt.Tooltip('hora:O'), alt.Tooltip('conexiones:Q')]
        ).properties(title=f'Conexiones por hora en zonas con 2G/4G — Top {N_COMUNAS} comunas')

        display(chart_top)
    else:
        print("Faltan columnas:", required - set(df_main.columns))
else:
    print("El DataFrame está vacío (placeholder).")


alt.Chart(...)

alt.Chart(...)

**Interpretaciones (completar):**
- [ ] **P1:** …
- [ ] **P2:** …
- [ ] **P3:** …


## Conclusiones y próximos pasos <a id="conclusiones"></a>
### Conclusión principal:

Se logró responder parcialmente a las preguntas planteadas:

**P1**: La distribución de conexiones por tecnología y comuna muestra todavía presencia de 2G, aunque el patrón horario se aprecia mejor de forma agregada o restringiendo a comunas con mayor volumen.

**P2**: Se identificaron dispositivos no compatibles con LTE/5G, pero los indicadores debieron inferirse indirectamente a partir del uso observado de tecnologías, ya que el dataset no incluía campos de compatibilidad explícitos.

**P3**: El análisis de migración indica que los usuarios que probaron 4G/5G no regresaron a 2G, aunque este resultado puede estar sesgado por la estructura y cobertura de los datos. En cuanto a la evolución de fallas, no fue posible analizar un histórico, pero sí se pudo visualizar la distribución horaria de conexiones en zonas con 2G/4G.

### Limitaciones:

**Cobertura temporal**: el dataset solo incluye un día de registros (2025-05-16), lo cual impide analizar tendencias a lo largo del tiempo y limita la validez de conclusiones longitudinales.

**Variables disponibles**: no existen métricas continuas (ej. volumen de tráfico, duración de sesión, intensidad de señal), lo que limita la detección de outliers y la caracterización más fina de la calidad de servicio.

**Campos inferidos**: las variables de compatibilidad LTE/5G se dedujeron a partir del comportamiento observado, no de un atributo explícito en la dimensión de dispositivos.

**Posible sesgo**: el resultado de que ningún usuario vuelva a 2G tras usar 4G/5G puede deberse a la falta de registros 2G en la muestra, más que a un comportamiento real.

**Granularidad**: la gran cantidad de comunas genera gráficos ilegibles si no se filtra o agrega información (ej. Top N comunas).

### Próximos pasos:

Incorporar datasets con mayor rango temporal (semanas o meses) para permitir análisis de evolución real.

Agregar métricas continuas (tráfico, duración, señal) que permitan un EDA más robusto y la detección de anomalías.

Obtener de la dimensión de dispositivos información explícita de compatibilidad tecnológica (LTE/5G) para validar los indicadores inferidos.

Desarrollar visualizaciones dinámicas o dashboards (ej. en Tableau, Power BI o Dash/Streamlit) que permitan filtrar comunas, tecnologías y períodos sin saturar la gráfica.

Explorar segmentaciones adicionales: franjas horarias, tipos de dispositivo, operadoras, etc., para entender mejor patrones de uso y de migración tecnológica.

Realizar análisis comparativos “pre/post” de migración cuando existan datos históricos suficientes.
